In [12]:
import pickle

all_dict = pickle.load(open('results/all_dict.pkl', 'rb'))

noses = all_dict['noses']
widths = all_dict['widths']
heights = all_dict['heights']
imgs = all_dict['cropped_imgs']
segments = all_dict['cropped_segments']

In [13]:
import math 
standard_nose_y = max([nose[1] for nose in noses])
max_padded_height = -1 
for height, nose in zip(heights, noses): 
    current_height = height + (standard_nose_y - nose[1])
    if current_height > max_padded_height:
        max_padded_height = current_height

max_padded_height = math.ceil(max_padded_height)

# min pad to get nose at the center x 
padded_widths = [] 
for width, nose in zip(widths, noses): 
    padded_widths.append(width + abs(width // 2 - nose[0]))

max_padded_width = math.ceil(max(padded_widths))

In [14]:
import shapely 
from collections import defaultdict
standard_nose = (max_padded_width//2, standard_nose_y) # depending on the dataset 
translated_segments = defaultdict(list)

for nose, segment, height, width in zip(list(noses), segments, heights, widths):  
    xoff = standard_nose[0] - nose[0]
    yoff = standard_nose[1] - nose[1]
    for label, polygons in segment.items(): 
        for polygon in polygons:
            translated_polygon = shapely.affinity.translate(polygon, xoff=xoff, yoff=yoff)
            translated_segments[label].append(translated_polygon)

In [28]:
from shapely.geometry import Polygon
import numpy as np 

def create_face_heatmaps(max_width, max_height, segments, patch_size=(100,100)):
    num_y = math.ceil(max_height / patch_size[0])
    num_x = math.ceil(max_width / patch_size[1])

    face_heatmaps = dict() 
    err_polygons = list()
    err_current_rects = list()
    for label in segments.keys():
      face_heatmap = np.zeros((num_y, num_x))
      for i in range(num_y):  
          for j in range(num_x): 
              current_rect = Polygon([
                  [j*patch_size[1], i*patch_size[0]], 
                  [j*patch_size[1] + patch_size[1], i*patch_size[0]],
                  [j*patch_size[1] + patch_size[1], i*patch_size[0] + patch_size[0]],
                  [j*patch_size[1], i*patch_size[0] + patch_size[0]] 
              ]) 
              for polygon in segments[label]: 
                try: 
                  area = current_rect.intersection(polygon).area * 1. / current_rect.area
                except:
                  area = current_rect.intersection(polygon.buffer(0)).area * 1. / current_rect.area
                if area > 0:
                  face_heatmap[i][j] += 1
      face_heatmaps[label] = face_heatmap
    return face_heatmaps

patch_size = (100, 100)
face_heatmaps = create_face_heatmaps(max_padded_width, max_padded_height, translated_segments, patch_size=(100,100))

Input geom 1 is INVALID: Self-intersection at or near point 916.06441446502185 2187.901495653603 (916.06441446502185499 2187.9014956536029786)
<A>
POLYGON ((677.3371952379097820 1961.2728182306527742, 593.5875547829141397 2010.5723331177528053, 526.6437406180884864 2043.3455127105357860, 442.6146091672405873 2125.9771891821228564, 400.0410614501126929 2233.9573505868902430, 540.4313758662467535 2403.8071227553186873, 654.9526364380751602 2376.1949533829656502, 748.3455334868610862 2312.6900809584717535, 826.9740910065485195 2306.2062751091616519, 898.6397152554653758 2278.2347601707128888, 948.4582848491476170 2300.0818148259468217, 927.4497031814701131 2249.9038996661688543, 920.5466608383816265 2221.2735845232114116, 935.0714366567492561 2192.8229421633022866, 906.5609033691573586 2185.4407723987533245, 920.7862245491120348 2192.7031603079367414, 899.5380791707042363 2171.0956693634334442, 864.3640672507542604 2106.5127602406528240, 835.9733158185272259 2084.8453783684667542, 829.249

In [37]:
import plotly.express as px
import plotly.graph_objects as go 
fig = go.Figure()

buttons = [] 
visible_list = [False] * (len(face_heatmaps.keys()) + 1)

height, width = face_heatmaps[list(face_heatmaps.keys())[0]].shape
heatmap_standard_nose = [round(standard_nose[0] / patch_size[0]), round(standard_nose[1] / patch_size[1])]

x = np.arange(width)
y = np.array([None] * width)
y[heatmap_standard_nose[0]-1:heatmap_standard_nose[0]+1] = height - heatmap_standard_nose[1]


fig.add_trace(go.Scatter(x=x , y=y,
     line =dict(color='white',width=3)))

for idx, (label, heatmap_data) in enumerate(face_heatmaps.items()):
    fig.add_trace(go.Heatmap(z=face_heatmaps[label][::-1]))

current_visible_list = visible_list.copy()
current_visible_list[0] = True
current_visible_list[idx+1] = True
button = dict(
    method='update',
    label=label, 
    args=[{'visible': current_visible_list}]
)
buttons.append(button)

fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "left",
            buttons=buttons[::-1],
            pad={"l":70, "t": -10},
            showactive=True,
            x=0.11,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ], 
    width=500,
    height=600
)

# Add annotation
fig.update_layout(
    annotations=[
        dict(text="Segment type:", showarrow=False,
                            x=5, y=1.10, yref="paper", align="left")
    ]
)

fig.show()